# Predicting YCombinator Startup Performance

### Load dataset

In [2]:
import pandas as pd
import os
from fuzzywuzzy import process
from fuzzywuzzy import fuzz


/opt/miniconda3/lib/python3.12/site-packages/fuzzywuzzy/fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [11]:
badges = pd.read_csv('archive/badges.csv')
companies = pd.read_csv('archive/companies.csv')
founders = pd.read_csv('archive/founders.csv')
industries = pd.read_csv('archive/industries.csv')
prior_companies = pd.read_csv('archive/prior_companies.csv')
regions = pd.read_csv('archive/regions.csv')
schools = pd.read_csv('archive/schools.csv')
tags = pd.read_csv('archive/tags.csv')

badges['badge_num'] = badges.groupby('id').cumcount() + 1
df_pivot = badges.pivot(index='id', columns='badge_num', values='badge')
df_pivot.columns = [f'badge_{int(col)}' for col in df_pivot.columns]
badges = df_pivot.reset_index()

industries['industry_num'] = industries.groupby('id').cumcount() + 1
df_pivot = industries.pivot(index='id', columns='industry_num', values='industry')
df_pivot.columns = [f'industry_{int(col)}' for col in df_pivot.columns]
industries = df_pivot.reset_index()

tags['tag_num'] = tags.groupby('id').cumcount() + 1
df_pivot = tags.pivot(index='id', columns='tag_num', values='tag')
df_pivot.columns = [f'tag_{int(col)}' for col in df_pivot.columns]
tags = df_pivot.reset_index()

schools = schools.drop_duplicates(subset='hnid', keep='first')
ivy_league_schools = [
    'Harvard University',
    'Yale University',
    'Princeton University',
    'Columbia University',
    'University of Pennsylvania',
    'Dartmouth College',
    'Brown University',
    'Cornell University'
]

#TODO why all false?
def is_ivy_league(school_name):
    # Strip whitespace and use fuzzy matching
    school_name = str(school_name)
    school_name = school_name.strip().lower()
    # Get the best match and its score
    best_match, score = process.extractOne(school_name, ivy_league_schools, scorer=fuzz.ratio)
    return score >= 80

schools['is_ivy_league'] = schools['school'].apply(is_ivy_league)
faang_companies = [
    'Facebook',
    'Apple',
    'Meta',
    'Amazon',
    'Netflix',
    'Microsoft',
    'Google'
]
def is_faang(company_name):
    company_name = str(company_name)
    company_name = company_name.strip().lower()
    best_match, score = process.extractOne(company_name, faang_companies, scorer=fuzz.ratio)
    return score >= 80  # Threshold for matching

prior_companies['is_faang'] = prior_companies['company'].apply(is_faang)

founder_dfs = [founders, schools, prior_companies]
company_dfs = [badges, companies, industries, regions, tags]

for i, df in enumerate(company_dfs):
    if 'Unnamed: 0' in df.columns:
        df.drop(columns=['Unnamed: 0'], inplace=True)

merged_df = company_dfs[0]

for i, df in enumerate(company_dfs[1:], start=1):
    merged_df = pd.merge(merged_df, df, on='id', how='inner')

merged_df.rename(columns={'slug': 'company_slug'}, inplace=True)
merged_df = pd.merge(merged_df, founders, on='company_slug', how='inner')
#merged_df = pd.merge(merged_df, schools, on='hnid', how='inner')
#merged_df = pd.merge(merged_df, prior_companies, on='hnid', how='inner')

# Final merged DataFrame
print("Final merged DataFrame:")
prior_companies


Final merged DataFrame:


,hnid,company,is_faang
0,110,Khoj,False
1,110,Microsoft,True
2,110,Hillhacks,False
3,__JW__,Union54,False
4,__sy__,Seam,False
...,...,...,...
32607,zxie,NASA Jet Propulsion Laboratory,False
32608,zyro,Heroic Labs,False
32609,zyro,Masabi,False
32610,zyro,M&G,False


### List dataset variables 

### Visualize Dataset

### Create variables